<a href="https://colab.research.google.com/github/koreahong/caculate_conversion_rate/blob/main/%EC%A0%84%ED%99%98%EB%A5%A0_%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd

#데이터전처리

## 파일업로드하기
  - 실제 테스트환경에서는 str 타입으로 주어서 업로드 과정없이 split을 하여 사용했어야 함

In [12]:
txt_file = open("/gdrive/MyDrive/머신러닝_실전예제/토스코딩테스트/toss_log_codingtest.txt", 'r')
data = txt_file.read()
txt_file.close()

## txt파일을 DataFrame으로 변환하기

In [91]:
log = data.split()
df = [user_log.split(',') for user_log in log[1:]]
df = pd.DataFrame(df)

In [15]:
#컬럼명이 모두 기억이 안나서 필요했던 부분만 추출해서 컬럼명을 변경함
#유저아이디, 이벤트명, 운영체제명 추출
df = df[[1, 3, 7]]
df.columns = ['유저아이디', '이벤트명', '운영체제명']

In [16]:
df.head()

,유저아이디,이벤트명,운영체제명
0,8cff9a98-f743-4533-9d06-84cfcd0c6ac5,quiz_main,android
1,3ac1db4a-03ae-40d5-be75-238cb522c1e2,timeline_detail,android
2,2394a002-3d15-4dee-97a7-bf05934608d4,dashboard,android
3,19e5f3ec-0932-460f-bf68-33117de21055,timeline_main,android
4,9f7be1d1-9689-48fb-aa79-16762cbcc2fe,event_my_profile,android


In [17]:
print(len(df))

56747


#  

## 유저아이디, 이벤트명별로 중복제거

In [20]:
df_non_duplicate = df.drop_duplicates(['유저아이디', '이벤트명'])

In [22]:
#중복없이 정제해서 줬다는 것을 알 수 있음
len(df_non_duplicate)

56747

## 운영체제명별로 나눠 저장하기

In [23]:
ios_data = df[df['운영체제명'] == 'ios']
android_data = df[df['운영체제명'] == 'android']

# 분석

## 이벤트명별 카운트세기

In [37]:
#문제에서 주어진 과정 8개가 생각이 안나서 임의로 8개를 골라서 진행
event_type = ['home', 'payment', 'transfer', 'bank_register', 'account_register', 'account_agreement', 'payment_register', 'transfer_complete']
extract_col_ios = ios_data['이벤트명'].value_counts().to_frame().reset_index()

ios_result = extract_col_ios[extract_col_ios.apply(lambda x: x['index'] in event_type, axis=1)]
ios_result.columns = ['이벤트명', '유저수']

extract_col_android = android_data['이벤트명'].value_counts().to_frame().reset_index()
android_result = extract_col_android[extract_col_android.apply(lambda x: x['index'] in event_type, axis=1)]
android_result.columns = ['이벤트명', '유저수']

In [39]:
#추출된테이블 확인
android_result

,이벤트명,유저수
9,home,500
13,account_agreement,493
16,bank_register,486
30,account_register,469
38,transfer,460
42,payment,450
50,payment_register,438
58,transfer_complete,60


## 전환율 구하기

In [ ]:
ios_result['다음_이벤트_유저수'] = ios_result['유저수'].tolist()[1:] + [0]
android_result['다음_이벤트_유저수'] = android_result['유저수'].tolist()[1:] + [0]

In [ ]:
ios_result['전환율'] = round(ios_result['다음_이벤트_유저수'] / ios_result['유저수'] * 100).astype('int')
android_result['전환율'] = round(android_result['다음_이벤트_유저수'] / android_result['유저수'] * 100).astype('int')

In [89]:
results = []
results.append(['ios',*sorted([(event1, event2, str(percent) + '%') for event1, event2, percent in zip(ios_result['이벤트명'][:-1], ios_result['이벤트명'][1:], ios_result['전환율'][:-1])], key = lambda x:x[2])[0]])
results.append(['android',*sorted([(event1, event2, str(percent) + '%') for event1, event2, percent in zip(android_result['이벤트명'][:-1], android_result['이벤트명'][1:], android_result['전환율'][:-1])], key = lambda x:x[2])[0]])

In [90]:
for result in results:
  print(*result)

ios payment_register transfer_complete 15%
android payment_register transfer_complete 14%
